In [1]:
!gdown --id 1oXLMVnuYZ0tX-MzMG8N8lwMsC5-aM-nv --output anotacoes_especialistas.csv
!gdown --id 1wG0iOHehIWF38kEpGhuHfFRotBBMC9t_ --output CSV.zip
!unzip CSV.zip

Downloading...
From: https://drive.google.com/uc?id=1oXLMVnuYZ0tX-MzMG8N8lwMsC5-aM-nv
To: /content/anotacoes_especialistas.csv
2.61MB [00:00, 23.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wG0iOHehIWF38kEpGhuHfFRotBBMC9t_
To: /content/CSV.zip
100% 1.32M/1.32M [00:00<00:00, 20.6MB/s]
Archive:  CSV.zip
   creating: CSV/
  inflating: CSV/original.csv        
  inflating: CSV/val.csv             
  inflating: CSV/test.csv            
  inflating: CSV/train.csv           
  inflating: CSV/train_additional_data.csv  


In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.svm import SVC 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [3]:
import pandas as pd

train_index = pd.read_csv('/content/CSV/train.csv')
val_index = pd.read_csv('/content/CSV/val.csv')
dados = pd.read_csv('/content/anotacoes_especialistas.csv')
train = dados.iloc[list(train_index['original_index'].values), 10:]
val = dados.iloc[list(val_index['original_index'].values), 10:]
train = train[sorted(train.columns.tolist())]
val = val[sorted(val.columns.tolist())]

In [4]:
X_train = []
for amostra in train.values:
  row = []
  for i in range(0, len(amostra), 3):
    arr = np.array([amostra[i], amostra[i+1], amostra[i+2]])
    row.append(int(np.nanmean(arr)))
  X_train.append(row)
X_train = np.array(X_train)

X_val = []
for amostra in val.values:
  row = []
  for i in range(0, len(amostra), 3):
    arr = np.array([amostra[i], amostra[i+1], amostra[i+2]])
    row.append(int(np.nanmean(arr)))
  X_val.append(row)
X_val = np.array(X_val)

In [5]:
X_train.shape, X_val.shape

((1111, 10), (278, 10))

In [6]:
y_train = dados.iloc[list(train_index['original_index'].values), 9].values
y_val = dados.iloc[list(val_index['original_index'].values), 9].values

In [7]:
counter = Counter(y_train)  
max_val = float(max(counter.values()))  
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}    
class_weights 

{0.0: 1.3982300884955752, 1.0: 1.0, 2.0: 1.5906040268456376}

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
params = {'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'degree': [1, 2, 3, 4, 5], 'gamma': ['scale', 'auto']}
svm = SVC(class_weight=class_weights)
clf = GridSearchCV(svm, param_grid=params, scoring='balanced_accuracy', cv=5)
clf.fit(X_train, y_train)
svm = clf.best_estimator_
print(svm)

pred = svm.predict(X_val)

print(confusion_matrix(y_val, pred))

cm = confusion_matrix(y_val, pred)
cm = cm / cm.astype(np.float).sum(axis=1, keepdims=True)
balanced_acc = np.trace(cm) / 3
print('Acuracia balanceada na validacao:', balanced_acc)

SVC(C=0.1, break_ties=False, cache_size=200,
    class_weight={0.0: 1.3982300884955752, 1.0: 1.0, 2.0: 1.5906040268456376},
    coef0=0.0, decision_function_shape='ovr', degree=1, gamma='scale',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
[[40 29 19]
 [44 49 14]
 [35 33 15]]
Acuracia balanceada na validacao: 0.36440409044845484


In [ ]:
import pickle

pickle.dump(svm, open('svm.sav', 'wb'))